<a href="https://colab.research.google.com/github/divy16/divy_11/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install DeepForest Flask flask-ngrok opencv-python matplotlib

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.6/22.6 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.2/543.2 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.4/587.4 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 890.6/890.6 kB 45.4 MB/s eta 0:00:00
   ━━

In [2]:
!mkdir templates

In [3]:
!pip install -q pyngrok==5.0.0


  Preparing metadata (setup.py) ... done


In [4]:
with open('/content/templates/index.html', 'w') as f:
    f.write('''
    <!-- index.html -->
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>DeepForest Model</title>
    </head>
    <body>
        <h1>Upload Image</h1>
        <form action="/predict" method="post" enctype="multipart/form-data">
            <input type="file" name="image">
            <input type="submit" value="Predict">
        </form>
    </body>
    </html>
    ''')


In [5]:
with open('/content/templates/result.html', 'w') as f:
    f.write('''
    <!-- result.html -->
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Prediction Result</title>
    </head>
    <body>
        <h1>Prediction Result</h1>
        <!-- Display the number of trees -->
        <p>Number of Trees: {{ num_trees }}</p>
        <!-- Display the image with bounding boxes -->
        <img src="data:image/png;base64,{{ image_data }}" alt="Predicted Image">
    </body>
    </html>
    ''')


In [6]:
# app.py
from flask import Flask, render_template, request, jsonify
from flask_ngrok import run_with_ngrok
from deepforest import main
import os
import cv2
import base64
from io import BytesIO

app = Flask(__name__)
# get_ipython().system_raw('./ngrok http 5000 &')
# !curl -s http://localhost:4040/api/tunnels | python3 -c \
#     "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"


# Load the DeepForest model during app initialization
model = main.deepforest()
model.use_release()

@app.route('/')
def index():
    return render_template('/content/templates/index.html')

@app.route('/predict', methods=['POST'])
def predict():
    # Get image from the request
    image = request.files['image']

    # Process the image using the DeepForest model
    img = model.predict_image(path=image, return_plot=True)

    # Count the number of trees
    num_trees = len(img)

    # Draw bounding boxes on the image
    cv2.putText(img, "TOTAL TREES: " + str(num_trees), (3, 14), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 0))

    # Convert the image to base64 for embedding in HTML
    buffered = BytesIO()
    cv2.imwrite(buffered, img)
    image_data = base64.b64encode(buffered.getvalue()).decode("utf-8")

    # Display the predictions and image in the result template
    return render_template('/content/templates/result.html', num_trees=num_trees, image_data=image_data)

if __name__ == '__main__':
    app.run(host='localhost')


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Reading config file: /usr/local/lib/python3.10/dist-packages/deepforest/data/deepforest_config.yml


Downloading: "https://download.pytorch.org/models/retinanet_resnet50_fpn_coco-eeacb38b.pth" to /root/.cache/torch/hub/checkpoints/retinanet_resnet50_fpn_coco-eeacb38b.pth
100%|██████████| 130M/130M [00:00<00:00, 161MB/s]
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.


config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

Reading config file: /usr/local/lib/python3.10/dist-packages/deepforest/data/deepforest_config.yml


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


model.safetensors:   0%|          | 0.00/129M [00:00<?, ?B/s]

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://localhost:5000
INFO:werkzeug:Press CTRL+C to quit
